# Centrality of C-Suite Executives

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
stock_df = pd.read_csv('~/project_ascend/comp_centrality/Data/fin_ratios.csv')

/var/folders/42/0ktpxb410cg701991z265xdw0000gn/T/ipykernel_24857/1729493979.py:1: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  stock_df = pd.read_csv('~/project_ascend/comp_centrality/Data/fin_ratios.csv')


In [3]:
# Cleanup and format stock_df
stock_df.columns = map(str.lower, stock_df.columns)
stock_df = stock_df.rename(columns={'tic': 'ticker'})
stock_df['public_date'] = pd.to_datetime(stock_df['public_date'], format='%m/%d/%Y')
stock_df['year'] = stock_df['public_date'].dt.year

# Load the executive data into exec_df
exec_df = pd.read_csv('~/project_ascend/comp_centrality/Data/Dow30Bio.csv')
exec_df.columns = map(str.lower, exec_df.columns)


In [4]:
# Curently only data from 2016 onwards
stock_df = stock_df[stock_df['year'] >= 2016]


In [5]:
stock_df_copy = stock_df.copy()

### Freeman Betweenness Centrality

Value of importance based on the number of shortest paths between that node and other nodes in the network.

Given an graph with N nodes, represented by an N x N adjacency matrix A, and a vector b representing the betweenness centrality values for each node,

Calculate the shortest path lengths between all pairs of nodes using an algorithm such as Dijkstra's algorithm.

For each node i, count the number of shortest paths that pass through it, and divide by the total number of shortest paths between all pairs of nodes in the network:

$$b_i = \frac{1}{(N-1)(N-2)}\sum_{s \neq i \neq t} \frac{\sigma_{st}(i)}{\sigma_{st}}$$

where $\sigma_{st}$ is the total number of shortest paths between nodes s and t, and $\sigma_{st}(i)$ is the number of shortest paths between s and t that pass through node i.

### Eigenvector Centrality

Value from one axis of greatest commonality/centrality amongst the elements of our feature space. Think of it like the eigenvector associated with the largest eigenvalue of the adjacency matrix

Given an undirected graph with N nodes, represented by an N x N adjacency matrix A, and a vector x representing the centrality values for each node,

Normalize the adjacency matrix A by dividing each row by the sum of its elements, so that each row represents the probability of moving from one node to another through a random walk:

$$P_{ij} = \frac{A_{ij}}{\sum_k A_{ik}}$$

Calculate the eigenvector v of the normalized adjacency matrix A corresponding to the largest eigenvalue λ, using an iterative algorithm such as the power method:

$$A v = \lambda v$$

where $v$ is the eigenvector and $\lambda$ is the corresponding eigenvalue.

The eigenvector centrality values for each node are given by the corresponding elements of the eigenvector v:

$$c_i = v_i$$

In other words, the eigenvector centrality of a node is proportional to the sum of the eigenvector centrality values of its neighbors, weighted by the strength of the connections between them:

$$c_i = \sum_j A_{ij} \frac{c_j}{\sum_k A_{jk}}$$

### Closeness Centrality

This value is how easily it is able to reach any other node on the network. Essentially, the the reciprocal of the sum of the shortest distances from that node to all other nodes in the graph.

$$C_i = \frac{1}{\sum_{j\neq i} d_{ij}}$$

where $C_i$ is the closeness centrality of node $i$, $d_{ij}$ is the shortest path between nodes $i$ and $j$, and the sum is taken over all nodes $j$ that are not equal to node $i$.

This one didn't really help our model much.

In [6]:
# Create function to compute centrality

def create_graph_and_compute_centrality(exec_df, centrality_func):
    G = nx.Graph()

    for _, row in exec_df.iterrows():
        director_id = row['director_detail_id']
        company_id = row['company_id']
        G.add_node(director_id)
        other_directors = exec_df[(exec_df['company_id']==company_id) & (exec_df['director_detail_id']!=director_id)]['director_detail_id'].tolist()
        for other_director in other_directors:
            G.add_edge(director_id, other_director)

    return centrality_func(G)


In [7]:
betweenness_centrality = create_graph_and_compute_centrality(exec_df, nx.betweenness_centrality) # Betweenness centrality
closeness_centrality = create_graph_and_compute_centrality(exec_df, nx.closeness_centrality) # Closeness centrality
degree_centrality = create_graph_and_compute_centrality(exec_df, nx.degree_centrality) # Degree centrality
eigen_centrality = create_graph_and_compute_centrality(exec_df, nx.eigenvector_centrality) # Eigenvector centrality

In [8]:
def add_centrality_to_dataframe(df, centrality_dict, centrality_name):
    centrality_df = pd.DataFrame(list(centrality_dict.items()), columns=['director_detail_id', centrality_name])
    return df.merge(centrality_df, on='director_detail_id', how='left')

exec_df = add_centrality_to_dataframe(exec_df, betweenness_centrality, 'betweenness_centrality')
exec_df = add_centrality_to_dataframe(exec_df, eigen_centrality, 'eigenvector_centrality')
exec_df = add_centrality_to_dataframe(exec_df, closeness_centrality, 'closeness_centrality')
exec_df = add_centrality_to_dataframe(exec_df, degree_centrality, 'degree_centrality')

In [9]:
# aggregate centrality values
comp_centrality = exec_df.groupby(['year', 'ticker'])[['betweenness_centrality', 'eigenvector_centrality', 'closeness_centrality', 'degree_centrality']].mean().reset_index()

In [10]:
comp_centrality.head()

,year,ticker,betweenness_centrality,eigenvector_centrality,closeness_centrality,degree_centrality
0,2016,AAPL,0.017648,0.034512,0.324897,0.032494
1,2016,AIG,0.005334,0.026385,0.252304,0.032714
2,2016,AMGN,0.005564,0.004469,0.288048,0.026482
3,2016,AXP,0.006697,0.007273,0.298905,0.035219
4,2016,BA,0.008980,0.005654,0.298271,0.037666


#### Merge with stock_df data

In [11]:
merged_df = stock_df.merge(comp_centrality, on = ['year', 'ticker'], how = 'left')

In [12]:
merged_df = merged_df.dropna(subset=['betweenness_centrality', 'eigenvector_centrality', 'closeness_centrality', 'degree_centrality'])


In [13]:
merged_df.head()

,gvkey,permno,adate,qdate,public_date,capei,bm,evm,pe_op_basic,pe_op_dil,...,ptb,peg_trailing,divyield,ticker,cusip,year,betweenness_centrality,eigenvector_centrality,closeness_centrality,degree_centrality
1549,1300,10145,12/31/2014,09/30/2015,2016-01-31,25.062,0.250,11.342,17.086,17.258,...,4.356,0.727,2.31%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1550,1300,10145,12/31/2015,12/31/2015,2016-02-29,22.007,0.236,11.160,15.811,16.011,...,4.138,0.915,2.35%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1551,1300,10145,12/31/2015,12/31/2015,2016-03-31,24.078,0.236,11.160,17.480,17.701,...,4.528,1.012,2.12%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1552,1300,10145,12/31/2015,12/31/2015,2016-04-30,24.580,0.236,11.160,17.827,18.052,...,4.622,1.032,2.08%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1553,1300,10145,12/31/2015,03/31/2016,2016-05-31,25.144,0.220,10.913,17.352,17.566,...,4.609,1.061,2.09%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419


In [14]:
merged_df.shape

(4136, 71)

In [15]:
stock_df.columns

Index(['gvkey', 'permno', 'adate', 'qdate', 'public_date', 'capei', 'bm',
       'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf',
       'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe',
       'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity',
       'pretret_noa', 'pretret_earnat', 'gprof', 'equity_invcap',
       'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt',
       'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
       'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
       'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
       'debt_capital', 'de_ratio', 'sale_invcap', 'sale_equity', 'sale_nwc',
       'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'peg_trailing',
       'divyield', 'ticker', 'cusip', 'year'],
      dtype='object')

In [16]:
merged_df.to_csv('dow_centrality.csv', index=False)

## Contemporaneous Variables

In [17]:
import statsmodels.api as sm

In [18]:
# check the rows with ticker == 'HON'
merged_df[merged_df['ticker'] == 'HON']

,gvkey,permno,adate,qdate,public_date,capei,bm,evm,pe_op_basic,pe_op_dil,...,ptb,peg_trailing,divyield,ticker,cusip,year,betweenness_centrality,eigenvector_centrality,closeness_centrality,degree_centrality
1549,1300,10145,12/31/2014,09/30/2015,2016-01-31,25.062,0.250,11.342,17.086,17.258,...,4.356,0.727,2.31%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1550,1300,10145,12/31/2015,12/31/2015,2016-02-29,22.007,0.236,11.160,15.811,16.011,...,4.138,0.915,2.35%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1551,1300,10145,12/31/2015,12/31/2015,2016-03-31,24.078,0.236,11.160,17.480,17.701,...,4.528,1.012,2.12%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1552,1300,10145,12/31/2015,12/31/2015,2016-04-30,24.580,0.236,11.160,17.827,18.052,...,4.622,1.032,2.08%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
1553,1300,10145,12/31/2015,03/31/2016,2016-05-31,25.144,0.220,10.913,17.352,17.566,...,4.609,1.061,2.09%,HON,43851610,2016,0.004561,0.004019,0.249157,0.023419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,1300,10145,12/31/2021,06/30/2022,2022-08-31,25.984,0.170,19.309,21.542,21.764,...,6.390,NaN,2.07%,HON,43851610,2022,0.004403,0.004446,0.250355,0.024356
1629,1300,10145,12/31/2021,06/30/2022,2022-09-30,22.866,0.170,19.309,18.995,19.192,...,5.624,NaN,2.35%,HON,43851610,2022,0.004403,0.004446,0.250355,0.024356
1630,1300,10145,12/31/2021,06/30/2022,2022-10-31,27.940,0.170,19.309,23.210,23.451,...,6.871,NaN,1.92%,HON,43851610,2022,0.004403,0.004446,0.250355,0.024356
1631,1300,10145,12/31/2021,09/30/2022,2022-11-30,29.982,0.179,17.744,24.340,24.531,...,7.339,NaN,1.88%,HON,43851610,2022,0.004403,0.004446,0.250355,0.024356


In [19]:
merged_df['ticker'].unique()

array(['HON', 'AXP', 'AIG', 'AMGN', 'AAPL', 'VZ', 'BA', 'CAT', 'JPM',
       'CVX', 'KO', 'C', 'DIS', 'DWDP', 'DD', 'XOM', 'GE', 'GIS', 'GM',
       'GT', 'HPQ', 'HD', 'INTC', 'IBM', 'IP', 'JNJ', 'MRO', 'MCD', 'MRK',
       'MMM', 'BAC', 'NKE', 'OI', 'PFE', 'MO', 'PG', 'T', 'UNH', 'UTX',
       'RTX', 'WMT', 'WBA', 'WDC', 'MSFT', 'MAN', 'CSCO', 'X', 'AA',
       'DOW', 'TRV', 'GS', 'CRM', 'SE', 'V'], dtype=object)

In [20]:
# For each ticker, keep the row for each year that has a qdate in December
merged_df = merged_df.sort_values(['ticker', 'year', 'qdate'])
merged_df = merged_df.groupby(['ticker', 'year']).tail(1)

In [21]:
# fill missing values with 0
merged_df = merged_df.fillna(0)

## Lagged Centrality Effects & OLS

By lagging the centrality measures, we can establish the time precedence of centrality variables with respect to the ROA and ROE so we can better interpret the results as causal of nature. This also mitigates the probability of reverse causality where it could be possible to infer that firm performance could influence network centrality instead of the other way around.

In [22]:
# Create lagged variables
comp_centrality['year'] = comp_centrality['year'] + 1


In [23]:
# merge lagged centrality measures with stock_df
lagged_merged_df = stock_df.merge(comp_centrality, on = ['year', 'ticker'], how = 'left')
lagged_merged_df = lagged_merged_df.dropna(subset=['betweenness_centrality', 'eigenvector_centrality', 'closeness_centrality', 'degree_centrality'])

In [24]:
# maintain contemporaneous variables only
lagged_merged_df = lagged_merged_df[lagged_merged_df['year'] >= 2017]

# for each ticker, keep the row for each year that has a qdate in December
lagged_merged_df = lagged_merged_df.sort_values(['ticker', 'year', 'qdate'])
lagged_merged_df = lagged_merged_df.groupby(['ticker', 'year']).tail(1)

# fill missing values with 0
lagged_merged_df = lagged_merged_df.fillna(0)

In [25]:
# Create variables for OLS
lagged_X = lagged_merged_df[['betweenness_centrality', 'eigenvector_centrality', 'closeness_centrality']]
lagged_X = sm.add_constant(lagged_X)
lagged_y_roa = lagged_merged_df['roa']
lagged_y_roe = lagged_merged_df['roe']

# Regress lagged centralities on return on asset
lagged_model_roa = sm.OLS(lagged_y_roa, lagged_X).fit()
print(lagged_model_roa.summary())



                            OLS Regression Results                            
Dep. Variable:                    roa   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.239
Date:                Fri, 17 Mar 2023   Prob (F-statistic):             0.0226
Time:                        11:04:52   Log-Likelihood:                 336.26
No. Observations:                 295   AIC:                            -664.5
Df Residuals:                     291   BIC:                            -649.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

In [26]:
# Regress lagged centralities on return on equity
lagged_model_roe = sm.OLS(lagged_y_roe, lagged_X).fit()
print(lagged_model_roe.summary())

                            OLS Regression Results                            
Dep. Variable:                    roe   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.2499
Date:                Fri, 17 Mar 2023   Prob (F-statistic):              0.861
Time:                        11:04:52   Log-Likelihood:                -580.10
No. Observations:                 295   AIC:                             1168.
Df Residuals:                     291   BIC:                             1183.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

## Panel Data Regression

Panel Data Regression gives us advantages of both cross-sectional and time-series data, providing us with more info about how centrality affects differ across the different companies in our dataset. Essentially creating a dummy variable for each company, each with its own coefficient in the linear model. However, these coefficients are not represented in the output summary, only the coefficients of the dependent variable in question (ROE, ROA).

$$
\text{ROA/ROE} = \beta_0 + \beta_1 \cdot \text{Betweenness Centrality} + \beta_2 \cdot \text{Eigenvector Centrality} + \beta_3 \cdot \text{Closeness Centrality} + \beta_4 \cdot \text{Degree Centrality} + \alpha_1 \cdot D_1 + \alpha_2 \cdot D_2 + \cdots + \alpha_n \cdot D_n + \epsilon
$$

Where:

- $\text{ROA/ROE}$ is the dependent variable (Return on Assets or Return on Equity).
- $\beta_0$ is the overall intercept term.
- $\beta_1, \beta_2, \beta_3,$ and $\beta_4$ are the coefficients of the centrality measures.
- $\text{Betweenness Centrality}, \text{Eigenvector Centrality}, \text{Closeness Centrality},$ and $\text{Degree Centrality}$ are the independent variables representing network centrality measures.
- $\alpha_1, \alpha_2, \dots, \alpha_n$ are the coefficients of the dummy variables.
- $D_1, D_2, \dots, D_n$ are the dummy variables for each entity (company) except for one (to avoid the dummy variable trap).
- $\epsilon$ is the error term.


In [27]:
import pandas as pd
from linearmodels import PanelOLS, RandomEffects

# Set up panel data using lagged_merged_df
panel_data = lagged_merged_df[['ticker', 'year', 'roa', 'roe', 'betweenness_centrality', 'eigenvector_centrality', 'closeness_centrality', 'degree_centrality']]
panel_data['year'] = pd.to_datetime(panel_data['year'], format='%Y')
panel_data = panel_data.set_index(['ticker', 'year'])
panel_data = panel_data.groupby([pd.Grouper(level='ticker'), pd.Grouper(freq='Y', level='year')]).mean()
panel_data = panel_data.reset_index()
panel_data['year'] = panel_data['year'].dt.year
panel_data = panel_data.set_index(['ticker', 'year'])

# Regress lagged centralities on return on asset
fe_model = PanelOLS(panel_data['roa'], panel_data[['betweenness_centrality', 'eigenvector_centrality', 'closeness_centrality', 'degree_centrality']], entity_effects=True)
fe_result = fe_model.fit()
print(fe_result)


                          PanelOLS Estimation Summary                           
Dep. Variable:                    roa   R-squared:                        0.0618
Estimator:                   PanelOLS   R-squared (Between):             -40.413
No. Observations:                 295   R-squared (Within):               0.0618
Date:                Fri, Mar 17 2023   R-squared (Overall):             -36.300
Time:                        11:04:53   Log-likelihood                    606.45
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.9199
Entities:                          53   P-value                           0.0042
Avg Obs:                       5.5660   Distribution:                   F(4,238)
Min Obs:                       1.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             3.9199
                            

/var/folders/42/0ktpxb410cg701991z265xdw0000gn/T/ipykernel_24857/4009543801.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_data['year'] = pd.to_datetime(panel_data['year'], format='%Y')


In [28]:
# Regress lagged centralities on return on equity
re_model = RandomEffects(panel_data['roe'], panel_data[['betweenness_centrality', 'eigenvector_centrality', 'closeness_centrality']])
re_result = re_model.fit()
print(re_result)

                        RandomEffects Estimation Summary                        
Dep. Variable:                    roe   R-squared:                        0.0183
Estimator:              RandomEffects   R-squared (Between):              0.0900
No. Observations:                 295   R-squared (Within):               0.0005
Date:                Fri, Mar 17 2023   R-squared (Overall):              0.0426
Time:                        11:04:53   Log-likelihood                   -525.54
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1.8139
Entities:                          53   P-value                           0.1447
Avg Obs:                       5.5660   Distribution:                   F(3,292)
Min Obs:                       1.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             1.8139
                            